# Первое знакомство 😊

Чтобы загрузить исходные данные из CSV файла используем функционал Google Colab

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

In [8]:
from google.colab import files
uploaded = files.upload()

Saving dataset_cost.csv to dataset_cost.csv


Загружаем данные в датафрейм

In [11]:
import io
import pandas as pd

df = pd.read_csv(io.BytesIO(uploaded['dataset_cost.csv']))
df.head()

,appsflyer_id,install_date,user_cost
0,1652759784623-2460704185261443885,2022-05-17,0.642349
1,1655526699783-8574419046565448176,2022-06-18,0.157631
2,1654492019018-1278649829463899192,2022-06-06,0.997820
3,1653160250232-1033278339103232135,2022-05-21,0.872417
4,1649518196275-5026948722442160790,2022-04-09,0.898276


Посмотрим основные характеристики датасета

# Посмотрим что внутри 🔎


In [16]:
df.describe()

,user_cost
count,22208.000000
mean,0.497427
std,0.288605
min,0.000100
25%,0.245854
50%,0.497323
75%,0.746610
max,0.999976


Приводим к нужному типу данных данных столбцы с датами и посмотрим временной диапазон данных

In [18]:
df['event_date'] = pd.to_datetime(df['event_date'])
df['install_date'] = pd.to_datetime(df['install_date'])

print('min event date - ', df['event_date'].min())
print('max event date - ', df['event_date'].max())
print('min install date - ', df['install_date'].min())
print('max install date - ', df['install_date'].max())

KeyError: 'event_date'

# Вспомним, что такое когорты 😵

В качестве примера когортный отчет соберем для гео ▶ Russia

In [19]:
df_installs = df[df['country_code'] == 'RU'].groupby('install_date', as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'installs'})
df_installs

KeyError: 'country_code'

# Мы готовы смотреть инсталлы ⛳

In [20]:
import plotly.express as px

fig = px.line(df_installs, x = df_installs['install_date'], y = df_installs['installs'])
fig.show()

NameError: name 'df_installs' is not defined

Т.к. судя по минимальному значению event date данные в датасете ограничены 1 апреля 2022 г. - ограничим этой датой также и инсталлы

In [21]:
df_installs = df_installs[df_installs['install_date'] >= '2022-04-01']
df_installs

NameError: name 'df_installs' is not defined

Посмотрим на динамику инсталлов на скорректированных данных

In [22]:
fig = px.bar(df_installs, x = 'install_date', y = 'installs')
fig.show()

NameError: name 'df_installs' is not defined

Добавим атрибуцию в разрезе органика/закупка 🤼

In [23]:
df_installs_upd = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'RU')].groupby(['install_date', 'is_organic'], as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'installs'})
df_installs_upd

KeyError: 'country_code'

In [24]:
fig = px.line(df_installs_upd, x = df_installs_upd['install_date'], y = df_installs_upd['installs'], color=df_installs_upd['is_organic'])
fig.show()

NameError: name 'df_installs_upd' is not defined

Что можно сказать о закупке трафика из России? Попробуем посмотреть на трафик из США

In [25]:
df_installs_us = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'US')].groupby(['install_date', 'is_organic'], as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'installs'})
df_installs_us

KeyError: 'country_code'

In [26]:
fig_us = px.line(df_installs_us, x = df_installs_us['install_date'], y = df_installs_us['installs'], color=df_installs_us['is_organic'])
fig_us.show()

NameError: name 'df_installs_us' is not defined

Какую еще информацию об инсталлах мы можем узнать из таблицы? Ваши варианты welcome ⛹

# Золотые люди 🤑

Узнаем число платящих в дневных когортах и конверсию в платящего. Сначала посчитаем сколько пользователей сконвертилось в платящего на каждый день

In [27]:
df_payers = df[df['install_date'] >= '2022-04-01'].groupby(['appsflyer_id','install_date'], as_index = False).agg({'payments':'sum'})
df_payers = df_payers[df_payers['payments']>0]
df_payers = df_payers.groupby('install_date', as_index = False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'payers'})

df_payers

KeyError: "Column(s) ['payments'] do not exist"

In [28]:
df_installs_all = df[df['install_date'] >= '2022-04-01'].groupby('install_date', as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'installs'})
df_cr = pd.merge(df_installs_all, df_payers, on='install_date', how='left')
df_cr['payers'] = df_cr['payers'].fillna(0)
df_cr['cr'] = round(df_cr['payers'] / df_cr['installs'], 2)

df_cr

NameError: name 'df_payers' is not defined

In [29]:
fig_cr = px.line(df_cr, x='install_date', y='cr')
fig_cr.show()

NameError: name 'df_cr' is not defined

Что еще хотим узнать о платящих? 💸

# Собираем когортный отчет 🧑 в разрезе дня по США

In [ ]:
df_installs_cohort = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'US')].groupby('install_date', as_index=False).agg({'appsflyer_id':'nunique', 'revenue_usd':'sum'}).rename(columns = {'appsflyer_id':'installs'})
df_payers_cohort = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'US')].groupby(['appsflyer_id','install_date'], as_index = False).agg({'payments':'sum'})
df_payers_cohort = df_payers_cohort[df_payers_cohort['payments']>0]
df_payers_cohort = df_payers_cohort.groupby('install_date', as_index = False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'payers'})

df_cohort = pd.merge(df_installs_cohort, df_payers_cohort, on='install_date', how='left')
df_cohort['payers'] = df_cohort['payers'].fillna(0)
df_cohort['cr'] = round(df_cohort['payers'] / df_cohort['installs'], 2)
df_cohort['rpi'] = round(df_cohort['revenue_usd'] / df_cohort['payers'], 2)

df_cohort

NameError: name 'df' is not defined



```
# Выбран кодовый формат
```

# Домашнее задание 🥶

Расчитать и собрать когортный отчет c использованием Jupyter Notebook / Colab, результаты можно вывести также в Google Sheets.Рекомендуем когорты собирать в недельном разрезе, страны поделить на тиры (например, США, тир 1 = Европа и т.д., достаточно 3-4 тира).

- Installs
- Payers - число платящих
- Purchases - число покупок
- CR - процент платящих
- RPI - доход на одну установку
- Revenue - доход в абсолютном значении

Какие выводы можно сделать о когортах в игре? Дополните анализ несколькими визуализациями.

In [ ]:
# Функция для классификации стран по уровням
def classify_tier(country_code):
    if country_code == 'US':
        return 'Tier 1'
    elif country_code in ['GB', 'DE', 'FR', 'IT', 'ES', 'NL', 'SE', 'BE', 'AT', 'DK', 'FI', 'GR', 'IE', 'LU', 'NO', 'PT', 'CH']:
        return 'Tier 2'
    else:
        return 'Tier 3'

df = pd.read_csv(io.BytesIO(uploaded['dataset (1).csv']))

# Преобразование столбцов даты
df['event_date'] = pd.to_datetime(df['event_date'])
df['install_date'] = pd.to_datetime(df['install_date'])

# Применение функции classify_tier для классификации стран по уровням
df['tier'] = df['country_code'].apply(classify_tier)

# Извлечение install_week
df['install_week'] = df['install_date'].dt.to_period('W')

# Группировка данных по install_week и уровню для расчета требуемых показателей
cohort_data = (
    df[df['install_date'] >= '2022-04-01']
    .groupby(['install_week', 'tier'], as_index=False)
    .agg(
        installs=pd.NamedAgg(column='appsflyer_id', aggfunc='nunique'),
        revenue=pd.NamedAgg(column='revenue_usd', aggfunc='sum'),
        purchases=pd.NamedAgg(column='payments', aggfunc='sum')
    )
)

# Расчет плательщиков
df_payers = df[df['payments'] > 0].groupby(['install_week', 'tier'], as_index=False).agg(payers=pd.NamedAgg(column='appsflyer_id', aggfunc='nunique'))
cohort_data = pd.merge(cohort_data, df_payers, on=['install_week', 'tier'], how='left')
cohort_data['payers'].fillna(0, inplace=True)

# Расчет CR (коэффициента конверсии) и RPI (выручки за установку)
cohort_data['cr'] = (cohort_data['payers'] / cohort_data['installs']).round(2)
cohort_data['rpi'] = (cohort_data['revenue'] / cohort_data['installs']).round(2)

cohort_data.head()


,install_week,tier,installs,revenue,purchases,payers,cr,rpi
0,2022-03-28/2022-04-03,Tier 1,439,190.560000,44,34,0.08,0.43
1,2022-03-28/2022-04-03,Tier 2,110,39.188908,12,8,0.07,0.36
2,2022-03-28/2022-04-03,Tier 3,2385,323.763747,91,42,0.02,0.14
3,2022-04-04/2022-04-10,Tier 1,1067,478.890000,111,60,0.06,0.45
4,2022-04-04/2022-04-10,Tier 2,304,65.385337,12,6,0.02,0.22


In [ ]:
# Преобразование столбца install_week в строку
cohort_data['install_week'] = cohort_data['install_week'].astype(str)

# Построение графика количества установок по неделям и уровням
fig_installs = px.line(cohort_data, x='install_week', y='installs', color='tier', title='Количество установок в разбивке по неделям и уровням')
fig_installs.show()

# Построение графика выручки по неделям и уровням
fig_revenue = px.line(cohort_data, x='install_week', y='revenue', color='tier', title='Доход по неделям и уровням')
fig_revenue.show()



Домашнее задание:

In [ ]:
df_installs_cohort = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'UK' & df['country_code'] == 'FR' & df['country_code'] == 'DE')].groupby('install_date', as_index=False).agg({'appsflyer_id':'nunique', 'revenue_usd':'sum'}).rename(columns = {'appsflyer_id':'installs'})
df_payers_cohort = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'UK' & df['country_code'] == 'FR' & df['country_code'] == 'DE')].groupby(['appsflyer_id','install_date'], as_index = False).agg({'payments':'sum'})
df_payers_cohort = df_payers_cohort[df_payers_cohort['payments']>0]
df_payers_cohort = df_payers_cohort.groupby('install_date', as_index = False).agg({'appsflyer_id':'nunique', 'payments' : 'sum'}).rename(columns = {'appsflyer_id':'payers'})

df_cohort = pd.merge(df_installs_cohort, df_payers_cohort, on='install_date', how='left')
df_cohort['payers'] = df_cohort['payers'].fillna(0)
df_cohort['cr'] = round(df_cohort['payers'] / df_cohort['installs'], 2)
df_cohort['rpi'] = round(df_cohort['revenue_usd'] / df_cohort['payers'], 2)

df_cohort

TypeError: ignored

In [ ]:
df_payers_cohort